In [1]:
!pip install mtcnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.6 MB/s eta 0:00:00


In [2]:
!mkdir images

In [23]:
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/alireza.jpg -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/ali.jpg -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/mohsen.jpg -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/muhammad.jpg -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/1.jpg -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/srttu-class.png -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/m_wrapped.png -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/m.jpg -P images

--2023-06-28 07:50:27--  https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/alireza.jpg
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Alireza-Akhavan/deep-face-recognition/master/images/alireza.jpg [following]
--2023-06-28 07:50:27--  https://raw.githubusercontent.com/Alireza-Akhavan/deep-face-recognition/master/images/alireza.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39450 (39K) [image/jpeg]
Saving to: ‘images/alireza.jpg’

alireza.jpg         100%[===================>]  38.53K  --.-KB/s    in 0s      

2023-06-28 07:50:27 (87.4 MB/s) - ‘images/alireza.jpg’ s

In [4]:
!wget https://raw.githubusercontent.com/Alireza-Akhavan/deep-face-recognition/master/ArcFace.py

--2023-06-28 07:36:57--  https://raw.githubusercontent.com/Alireza-Akhavan/deep-face-recognition/master/ArcFace.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4043 (3.9K) [text/plain]
Saving to: ‘ArcFace.py’

ArcFace.py          100%[===================>]   3.95K  --.-KB/s    in 0s      

2023-06-28 07:36:57 (35.4 MB/s) - ‘ArcFace.py’ saved [4043/4043]



In [5]:
import ArcFace
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing import image
import numpy as np
%matplotlib inline

In [6]:
model = ArcFace.loadModel()
face_detector = MTCNN()

arcface_weights.h5  will be downloaded to  /root/.deepface/weights/arcface_weights.h5
Access denied with the following error:
pre-trained weights could not be loaded.
You might try to download it from the url  https://drive.google.com/uc?id=1LVB3CdVejpmGHM28BpqqkbZP5hDEcdZY  and copy to  /root/.deepface/weights/arcface_weights.h5  manually



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1LVB3CdVejpmGHM28BpqqkbZP5hDEcdZY 



In [7]:
def detect_face(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #mtcnn expects RGB but OpenCV read BGR
    detections = face_detector.detect_faces(img_rgb)
    detection = detections[0]
    x, y, w, h = detection["box"]
    detected_face = img[int(y):int(y+h), int(x):int(x+w)]
    return detected_face

In [8]:
def preprocess_face(img, target_size=(112,112)):
    img = cv2.imread(img)
    img = detect_face(img)
    img = cv2.resize(img, target_size)
    img_pixels = image.img_to_array(img)
    img_pixels = np.expand_dims(img_pixels, axis = 0)
    img_pixels /= 255 #normalize input in [0, 1]
    return img_pixels

In [9]:
def img_to_encoding(path):
    img = preprocess_face(path)
    return model.predict(img)[0]

In [24]:
database = {}

database["alireza"] = img_to_encoding("./images/alireza.jpg")
database["ali"] = img_to_encoding("./images/ali.jpg")
database["mohsen"] = img_to_encoding("./images/mohsen.jpg")
database["muhammad"] = img_to_encoding("./images/muhammad.jpg")

1/1 [==============================] - 0s 203ms/step


In [13]:
def EuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [14]:
verification_threshhold = 4.4

In [25]:
def verify(image_path, identity, database):
    encoding = img_to_encoding(image_path)

    dist = EuclideanDistance(encoding, database[identity])

    if dist < verification_threshhold:
        print("It's " + str(identity) + ", IDENTITY CONFIRMED")
    else:
        print("It's not " + str(identity) + ", UNRECOGNIZABLE FACE")

    return dist

In [26]:
verify("images/1.jpg", "alireza", database)

1/1 [==============================] - 0s 200ms/step
It's not alireza, UNRECOGNIZABLE FACE


24.07007

In [27]:
verify("images/ali.jpg", "alireza", database)

1/1 [==============================] - 0s 360ms/step
It's not alireza, UNRECOGNIZABLE FACE


16.424583

In [28]:
def who_is_it(image_path, database):

    encoding = img_to_encoding(image_path)

    min_dist = 1000
    for (name, db_enc) in database.items():
        dist = EuclideanDistance(encoding, db_enc)

        if min_dist > dist:
            min_dist = dist
            identity = name


    if min_dist > verification_threshhold:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))

    return min_dist, identity

In [29]:
who_is_it("images/srttu-class.png", database)

1/1 [==============================] - 0s 203ms/step
Not in the database.


(15.668687, 'alireza')